In [2]:
import os
import glob

import nibabel
import numpy as np

import torch
import torch.nn as nn



/home/song99/anaconda3/envs/medsegdiff/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data_path = "/home/song99/ws/datasets/AMOS" 
images = glob.glob(os.path.join(data_path, "labelsVa/*"))

In [5]:
num_classes = 14
coords = []

for image in images:
    image = nibabel.load(image).get_fdata()
    image = torch.from_numpy(image)
    centroids = [None for _ in range(num_classes)]
    for i in torch.unique(image):
        i = int(i.item())
        if i == 0: continue
        x, y, z = torch.where(image == i)
        centroids[i] = torch.stack([torch.mean(x.float()), torch.mean(y.float()), torch.mean(z.float())])
    
    coords.append(centroids)
    # print("="*100)
for centroids in coords:
    for i in range(1, num_classes):
        for j in range(i+1, num_classes):
            if centroids[i] is None or centroids[j] is None:
                continue
            vec1 = centroids[i]
            vec2 = centroids[j]
            
            cos_angle = torch.dot(vec1, vec2) / (torch.norm(vec1) * torch.norm(vec2))
            angle = torch.acos(cos_angle) #  * 180 / np.pi  # 각도를 라디안에서 도로 변환
            print(f"Angle between class {i} and class {j}: {angle} degrees")


Angle between class 1 and class 2: 0.30217382311820984 degrees
Angle between class 1 and class 3: 0.04397190734744072 degrees
Angle between class 1 and class 5: 0.21014031767845154 degrees
Angle between class 1 and class 6: 0.43590739369392395 degrees
Angle between class 1 and class 7: 0.2033926397562027 degrees
Angle between class 1 and class 8: 0.1958967000246048 degrees
Angle between class 1 and class 9: 0.27623996138572693 degrees
Angle between class 1 and class 10: 0.29597291350364685 degrees
Angle between class 1 and class 11: 0.2116854339838028 degrees
Angle between class 1 and class 12: 0.24554210901260376 degrees
Angle between class 1 and class 13: 0.13776353001594543 degrees
Angle between class 2 and class 3: 0.2598101496696472 degrees
Angle between class 2 and class 5: 0.09721299260854721 degrees
Angle between class 2 and class 6: 0.13380374014377594 degrees
Angle between class 2 and class 7: 0.1029222384095192 degrees
Angle between class 2 and class 8: 0.10629918426275253 d

In [6]:
import pandas as pd
import torch
import numpy as np
import nibabel

num_classes = 14
coords = []
all_angles = []  # 모든 이미지의 클래스 간 각도를 저장할 리스트

# 이미지 로딩 및 중심점 계산
for image in images:
    image = nibabel.load(image).get_fdata()
    image = torch.from_numpy(image)
    centroids = [None for _ in range(num_classes)]
    for i in torch.unique(image):
        i = int(i.item())
        if i == 0: continue
        x, y, z = torch.where(image == i)
        centroids[i] = torch.stack([torch.mean(x.float()), torch.mean(y.float()), torch.mean(z.float())])
    
    coords.append(centroids)
    
    image_angles = []  # 한 이미지의 클래스 간 각도를 저장할 리스트
    for i in range(1, num_classes):
        for j in range(i+1, num_classes):
            if centroids[i] is None or centroids[j] is None:
                angle = np.nan  # 정보가 없으면 NaN으로 저장
            else:
                vec1 = centroids[i]
                vec2 = centroids[j]
                
                cos_angle = torch.dot(vec1, vec2) / (torch.norm(vec1) * torch.norm(vec2))
                angle = torch.acos(cos_angle).item()  # 라디안으로 저장
                
            image_angles.append(angle)
            
    all_angles.append(image_angles)

# 판다스 데이터프레임으로 변환 및 CSV로 저장
columns = [f"angle_{i}_{j}" for i in range(1, num_classes) for j in range(i+1, num_classes)]
df = pd.DataFrame(all_angles, columns=columns)
df.to_csv("class_angles.csv", index=False)
